# Research markdown generator for academicpages

Takes a TSV of research with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `research.py`. Run either from the `markdown_generator` folder after replacing `research.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/research/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat research.tsv

pub_date	title	field	venue	excerpt	citation	url_slug	paper_url
2021-05-31	Embedded Dense Neural Networks for Battery Cyclability Prediction on Automotive Microcontroller Devices	Machine Learning	Harvard Library Office for Scholarly Communication	This paper explores the development of a testing methodology in the context of accurately predicting future battery performance using machine learning. In particular, I propose a Dense Neural Network Model that can predict battery life using data from the first 50 cycles in the battery life.	Rotaru, A. (2021). "Embedded Dense Neural Networks for Battery Cyclability Prediction on Automotive Microcontroller Devices" <i>Harvard Library Office for Scholarly Communication</i>	embedded-dnn-battery	https://dash.harvard.edu/handle/1/37368547
2016-02-16	The effects of Caloric restriction on the Biological Indices of Homeostasis	Pathophysiology, Medicine	MoldSEF National Science and Engineering Fair	  This paper studies the benefits of a caloric restrict

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
! pip install -U pandas
import pandas as pd

/bin/bash: pip: command not found


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
research = pd.read_csv("research.tsv", sep="\t", header=0)
research


,pub_date,title,field,venue,excerpt,citation,url_slug,paper_url
0,2021-05-31,Embedded Dense Neural Networks for Battery Cyc...,Machine Learning,Harvard Library Office for Scholarly Communica...,This paper explores the development of a testi...,"Rotaru, A. (2021). ""Embedded Dense Neural Netw...",embedded-dnn-battery,https://dash.harvard.edu/handle/1/37368547
1,2016-02-16,The effects of Caloric restriction on the Biol...,"Pathophysiology, Medicine",MoldSEF National Science and Engineering Fair,This paper studies the benefits of a caloric...,"Rotaru, A., Tataru, E. (2016). ""The effects of...",caloric-restriction,https://adrianarotaru.github.io/files/caloric.pdf
2,2017-04-01,Angiotensin 1-7 blunts in vitro induced acute ...,Cardiology,"European Journal of Heart Failure, 2017; <i> T...",Angiotensin 1-7 (Ang 1-7) comprises consistent...,"Cobet, V., Rotaru, A., et al., (2017). Angiote...",angiotensin,https://repository.usmf.md/handle/20.500.12710...
3,2018-05-01,Vanhoutte and Bowditch phenomena in heart fail...,Cardiology,"European Journal of Heart Failure, 2018","Evaluation of both 15,16-epoxyeicosatrien (1...","Tacu, L., Rotaru, A., et al. (2018). Vanhoutte...",bowditch,https://repository.usmf.md/handle/20.500.12710...
4,2017-04-01,Coronary response in doxorubicin-induced cardi...,Cardiology,"European Journal of Heart Failure, 2017; <i> T...",Coronary reserve and reactivity traits regardi...,"Cobet, V., Rotaru, A., et al. (2017). Coronary...",coronary,https://repository.usmf.md/handle/20.500.12710...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in research.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: research"""
    
    md += """\npermalink: /paper/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\nfield: '" + item.field + "'"
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
   
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download abstract here](" + item.paper_url + ")\n" 
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_research/" + md_filename, 'w') as f:
        f.write(md)

These files are in the research directory, one directory below where we're working from.

In [6]:
!ls ../_research/

2016-02-16-caloric-restriction.md  2018-05-01-bowditch.md
2017-04-01-angiotensin.md          2021-05-31-embedded-dnn-battery.md
2017-04-01-coronary.md
